In [12]:
from options.cdxbasics.cached import version, cached, Cache
# the function f,g are not cached but have versions
@version("0.0.1")
def f(x,y):
    return x*y    
@version("0.0.2", dependencies=[f])
def g(x,y):
    return f(-x,y)

# the cached function 'my_func' depends on g and therefore also on f
@cached("0.0.3", dependencies=[g])
def my_func( x,y, cache=None ):
    return g(2*x,y)

# the casched function 'my_big_func' depends on 'my_func' and therefore also on g,f
@cached("0.0.4", dependencies=[my_func])
def my_big_func(x,y,z, cache=None ):
    r = my_func(x,y,cache=cache)
    return r*z

# test versioning
print("Version", my_big_func.version) # --> 0.0.4 { my_func: 0.0.3 { g: 0.0.2 { f: 0.0.1 } } }

# function call without caching
r = my_big_func(2,3,4)                # does not generate a cache: 'cache' argument not provided

# delete existing caches
print("\nDelete existing cache")
cache = Cache(cache_mode="clear")     # path defaults to !/.cached (e.g. tempdir/.cached)
r = my_big_func(2,3,4,cache=cache)    # generates the cache for my_big_func and my_func 

# test caching
print("\nGenerate new cache")
cache = Cache()                       # path defaults to !/.cached (e.g. tempdir/.cached)
r = my_big_func(2,3,4,cache=cache)    # generates the cache for my_big_func and my_func 
print("\nReading cache")
r = my_big_func(2,3,4,cache=cache)    # reads cache for my_big_func

# update
print("\nUpdating cache")
cache_u = Cache(cache_mode="update")
r = my_big_func(2,3,4,cache=cache_u)  # updates the caches for my_big_func, my_func
print("\nReading cache")
r = my_big_func(2,3,4,cache=cache)    # reads cache for my_big_func

# update only top level cache
print("\nUpdating only 'my_big_func'")
cache_lu = Cache(cache_mode="on", update=[my_big_func] )
r = my_big_func(2,3,4,cache=cache_lu) # updates the cache for my_big_func using the cache for my_func
print("\nReading cache")
r = my_big_func(2,3,4,cache=cache)    # reads cached my_big_func

Version 0.0.4 { my_func: 0.0.3 { g: 0.0.2 { f: 0.0.1 } } }

Delete existing cache
00: Deleted existing 'my_big_func' cache C:/Users/hansb/AppData/Local/Temp/.cache/my_big_func_6ac240bc128ec33ca37c17c5aab243e46b976893ccf0c40a.pck
01:   Deleted existing 'my_func' cache C:/Users/hansb/AppData/Local/Temp/.cache/my_func_47317c662192f51fddd527cb89369f77c547fc58cca962d7.pck

Generate new cache
01:   Wrote 'my_func' cache C:/Users/hansb/AppData/Local/Temp/.cache/my_func_47317c662192f51fddd527cb89369f77c547fc58cca962d7.pck
00: Wrote 'my_big_func' cache C:/Users/hansb/AppData/Local/Temp/.cache/my_big_func_6ac240bc128ec33ca37c17c5aab243e46b976893ccf0c40a.pck

Reading cache
00: Successfully read cache for 'my_big_func' from 'C:/Users/hansb/AppData/Local/Temp/.cache/my_big_func_6ac240bc128ec33ca37c17c5aab243e46b976893ccf0c40a.pck'

Updating cache
00: Deleted existing 'my_big_func' cache C:/Users/hansb/AppData/Local/Temp/.cache/my_big_func_6ac240bc128ec33ca37c17c5aab243e46b976893ccf0c40a.pck
01:   D